# IEEE-CIS Fraud Detection -- Model [Ensemble]

In [1]:
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

In [4]:
from ml import simple

## Blending

In [13]:
sub = simple.Blend(['./Output/LGBM_model_1.csv', './Output/safe_box.csv'], 
                   'TransactionID', 'isFraud', './Output/blend01.csv')

In [14]:
sub = simple.Blend(['./Output/blend01.csv', './Output/XGB_model_1.csv'], 
                   'TransactionID', 'isFraud', './Output/blend02.csv')

In [15]:
sub = simple.Blend(['./Output/blend02.csv', './Output/random_forest_1.csv'], 
                   'TransactionID', 'isFraud', './Output/blend03.csv')

## Reference
- https://www.kaggle.com/smerllo/identify-unique-cards-id